In [37]:
from pyscf import gto, scf, cc

a = 2 # 2aB
nH = 20
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="6-31g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
mf.kernel()

mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)


nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Thu Feb  5 17:25:41 2026
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 20
[INPUT] num. electrons = 20
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 B

np.float64(-0.34769530196177206)

In [38]:
from pyscf import lo
import numpy as np
nocc = np.count_nonzero(mf.mo_occ)
orbocc = mf.mo_coeff[:,nfrozen:nocc]
lo_coeff  = lo.PipekMezey(mol, orbocc).kernel()

frag_lolist = [[i] for i in range(nocc)]



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 4.0867237175129  delta_f= 4.08672  |g|= 0.993069  2 KF 6 Hx
macro= 2  f(x)= 4.0904216412529  delta_f= 0.00369792  |g|= 0.000790692  2 KF 5 Hx
macro= 3  f(x)= 4.0904216412544  delta_f= 1.43174e-12  |g|= 2.3969e-05  1 KF 1 Hx
macro X = 3  f(x)= 4.0904216412544  |g|= 2.3969e-05  6 intor 5 KF 12 Hx


In [ ]:
from collections.abc import Iterable
from pyscf.lno import lnoccsd
# from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 1e-5
run_frg_list = [0]
chol_cut = 1e-5

mlno = lnoccsd.LNOCCSD(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
print(f'number of fragment {nfrag}')
print(f'fragment {frag_lolist}')

if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)

[0.0001, 1e-05]
number of fragment 1
fragment [[0]]
LO occ proj: 1 active | 0 standby | 9 orthogonal
LO vir proj: 0 active | 0 standby | 30 orthogonal


In [40]:
lno_thresh

[0.0001, 1e-05]

In [41]:
mol = mf.mol
nocc = np.count_nonzero(mf.mo_occ)
actfrag = np.array([i for i in range(mol.nao) if i not in lno_frozen])
frzocc = np.array([i for i in range(nocc) if i in lno_frozen])
actocc = np.array([i for i in range(nocc) if i in actfrag])
actvir = np.array([i for i in range(nocc,mol.nao) if i in actfrag])
nfrzocc = len(frzocc)
nactocc = len(actocc)
nactvir = len(actvir)
nactorb = len(actfrag)

ncas = nactorb
ncore = nfrzocc
nelec = nactocc*2

print(nelec,ncas,ncore)

12 24 4


In [59]:
mo_occ = mlno.mo_occ
lno_frozen, maskact = lnoccsd.get_maskact(lno_frozen, mo_occ.size)

mf.max_memory = 0
mcc = lnoccsd.CCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=4)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

mcc.verbose = 4
from pyscf.lno import lnoccsd
(elcorr_pt2, elcorr_cc, elcorr_cc_t), frag_msg \
    = lnoccsd.impurity_solve(mcc, lno_coeff, uocc_loc, mo_occ, maskact, eris, log=None,
                             ccsd_t=False, verbose_imp=5)

Init t2, MP2 energy = -10.8073526019486  E_corr(MP2) -0.136611615535208

******** <class 'pyscf.lno.lnoccsd.MODIFIED_CCSD'> ********
CC2 = 0
CCSD nocc = 6, nmo = 24
frozen orbitals [ 0  1  2  3 28 29 30 31 32 33 34 35 36 37 38 39]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 0 MB (current use 786 MB)
Init E_corr(MODIFIED_CCSD) = -0.136611615535208
cycle = 1  E_corr(MODIFIED_CCSD) = -0.163245578596127  dE = -0.0266339631  norm(t1,t2) = 0.0739705
cycle = 2  E_corr(MODIFIED_CCSD) = -0.173893521966102  dE = -0.0106479434  norm(t1,t2) = 0.0442132
cycle = 3  E_corr(MODIFIED_CCSD) = -0.181134070248931  dE = -0.00724054828  norm(t1,t2) = 0.0259414
cycle = 4  E_corr(MODIFIED_CCSD) = -0.184640307970143  dE = -0.00350623772  norm(t1,t2) = 0.014261
cycle = 5  E_corr(MODIFIED_CCSD) = -0.184175429418278  dE = 0.000464878552  norm(t1,t2) = 0.00482109
cycle = 6  E_corr(MODIFIED_CCSD) = -0.184117798555936

In [43]:
frag_msg

'E_corr(MP2) = -0.0259204696136587  E_corr(CCSD) = -0.0348621223712252  E_corr(CCSD(T)) = 0'

In [44]:
print(elcorr_pt2, elcorr_cc, elcorr_cc_t)

-0.025920469613658695 -0.0348621223712252 0.0


In [66]:
print(mcc._scf._eri)

None


In [64]:
if getattr(mcc._scf, 'with_df', None):
    print("True")

True


In [48]:
imp_eris = mcc.ao2mo()

In [53]:
ovov = imp_eris.ovov
ovov = np.array(ovov)
print(ovov.shape)
ovov = ovov.reshape(nactocc*nactvir,nactocc*nactvir)
print(ovov.shape)

(6, 18, 6, 18)
(108, 108)


In [92]:
imp_eris.ovov.shape

(6, 18, 6, 18)

In [91]:
imp_eris.vvvv.shape

(171, 171)

In [80]:
print('# Calculating Effective Active Space One-electron Integrals')
mo_coeff = lno_coeff
frozen = lno_frozen
mol = mf.mol
nocc = np.count_nonzero(mf.mo_occ)
actfrag = np.array([i for i in range(mol.nao) if i not in frozen])
frzocc = np.array([i for i in range(nocc) if i in frozen])
actocc = np.array([i for i in range(nocc) if i in actfrag])
actvir = np.array([i for i in range(nocc,mol.nao) if i in actfrag])
nfrzocc = len(frzocc)
nactocc = len(actocc)
nactvir = len(actvir)
nactorb = len(actfrag)

ncas = nactorb
ncore = nfrzocc
nelec = nactocc*2
# h1e, enuc = h1e_ras(mf, mo_coeff, ncas, ncore)
mo_act = mo_coeff[:,actfrag]

# Calculating Effective Active Space One-electron Integrals


In [83]:
mo_act.shape

(40, 24)

In [ ]:
from pyscf.ao2mo import _ao2mo
from pyscf import df,lib

mo_coeff = np.asarray(imp_eris.mo_coeff, order='F')
nocc = imp_eris.nocc
nao, nmo = mo_coeff.shape
nvir = nmo - nocc
nvir_pair = nvir*(nvir+1)//2

naux = mycc._scf.with_df.get_naoaux()
Lpq_full = np.empty((naux,nmo,nmo))
ijslice = (0, nmo, 0, nmo)
Lpq = None
p1 = 0

for eri1 in mcc._scf.with_df.loop():
    Lpq = _ao2mo.nr_e2(eri1, mo_act, ijslice, aosym='s2', out=Lpq).reshape(-1,nmo,nmo)
    p0, p1 = p1, p1 + Lpq.shape[0]
    print(eri1.shape)
    print(Lpq.shape)
    Lpq_full[p0:p1] = Lpq


(240, 820)
(240, 24, 24)
(220, 820)
(220, 24, 24)


In [105]:
Lpq_full.shape

(460, 24, 24)

In [109]:
np.max(Lpq_full - Lpq_full.transpose(0,2,1))

np.float64(9.43689570931383e-16)

In [106]:
naux

460

In [100]:
print(Lpq.shape)

(220, 24, 24)


In [110]:
from pyscf import df
dferi = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
print(dferi.shape)

(460, 820)


In [116]:
from ad_afqmc import pyscf_interface
from pyscf import df,lib
print("# Composing AO ERIs from DF basis")
dferi = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
dferi = lib.unpack_tril(dferi).reshape(dferi.shape[0], -1)
dferi = dferi.reshape((-1, mol.nao, mol.nao))
dferi_mo = lib.einsum('pr,grs,sq->gpq',mo_act.T,dferi,mo_act)
eri = lib.einsum('lpq,lrs->pqrs', dferi_mo, dferi_mo, optimize='optimal')


# Composing AO ERIs from DF basis


In [117]:
print(eri.shape)

(24, 24, 24, 24)


In [118]:
eri = lib.pack_tril(eri,axis=0)
print(eri.shape)

(300, 24, 24)


In [120]:
eri = lib.pack_tril(eri)
print(eri.shape)

(300, 300)


In [121]:
print("# Composing active space MO ERIs from AO ERIs")
# eri = eri.reshape(ncas**2,ncas**2)
print("# Decomposing MO ERIs to Cholesky vectors")
print(f"# Cholesky cutoff is: {chol_cut}")
chol = pyscf_interface.modified_cholesky(eri,max_error=chol_cut)
# chol = chol.reshape(-1,ncas,ncas)

# Composing active space MO ERIs from AO ERIs
# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-05


In [122]:
print(chol.shape)

(91, 300)


In [86]:
chol.shape

(91, 24, 24)

In [123]:
chol = lib.unpack_tril(chol)
chol.shape

(91, 24, 24)

In [111]:
e2int = lib.einsum('gpr,gqs->prqs', Lpq_full, Lpq_full)

In [124]:
e2int2 = lib.einsum('gpr,gqs->prqs', chol, chol)

In [125]:
np.max(e2int - e2int2)

np.float64(9.850088451725594e-06)